<a href="https://colab.research.google.com/github/ki-seoultech/DACON-SamSung-SCPC-2025/blob/main/%EC%82%BC%EC%84%B1_AI_%EC%B1%8C%EB%A6%B0%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
<2025 삼성 scpc대회 참가작>
vqa형식의 4지선다 분류문제
첫 외부대회 도전기 → 결과: 리더보드 185등, 성능0.27
성능은 분석하는게 의미없을정도로 다소 형편없지만 짧은기간동안 혼자 힘으로 외부대회를 경험했다는것을 도전의 발판으로 삼음.

<이번대회에서 새롭게 알게된 경험과 지식>
1.외부데이터를 직접 가져와서 모델학습
1-1. AOKVQA 외부데이터
1-2. VQAV2  외부데이터  & COCO 2014 이미지

2.pretrained model 불러오기
2-1. clip2 모델
2-1. blip2 모델
2-3. convnext 모델
실질적으로 유의미한 성능을 낸건 clip2 모델

3.학습데이터(삼성에서 주어진 소규모 60개 데이터) augmentation
3-1. 이미지 증강 (Image Augmentation)
Random Horizontal Flip
→ 50% 확률로 이미지를 좌우 반전
→ 모델이 방향성에 덜 민감해져서 일반화 성능 향상

3-2. Color Jitter
→ 밝기(brightness), 대비(contrast), 채도(saturation), 색조(hue)를 무작위로 조정
→ 조명이나 색상 차이에 강한 모델 학습

3-3. Resize (224x224)
→ CLIP 입력 크기에 맞게 이미지 스케일을 표준화

3-4. 선택지 셔플 (Choice Shuffling)
증강된 샘플(aug_idx > 0)에서는 보기 순서를 무작위로 섞음

정답 텍스트가 섞인 뒤에도 그대로 정답 인덱스를 유지

모델이 “위치”에 의존하지 않고 텍스트 의미로 정답을 찾게 만듦

3-5. 증강 배수 (Augment Ratio)
원본 하나당 4배로 확장 → 같은 질문도 다른 색상·회전·보기에 대해 여러 번 학습

결과적으로 240개의 삼성 데이터

4.외부데이터 데이터비율 ensemble
4-1.AOK 확장 조합
(240/480/120)	반드시 포함	reasoning 성능 보강, BLIP2와 상보성. 성능·다양성 모두 확보

4-2.VQA 확장 조합
(240/240/360)	포함 권장	CLIP 특화 강화, factual Q에 강한 베이스. 성능 극대화 시도

4-3.균형형 조합
(240/300/300)	유지력 중심	중간 모델. 위 2개 조합 간 중첩 문제 완화 역할


<이번 프로젝트에서 겪은 시행착오>
문제상황 → 해결방안
학습 데이터 부족
본 삼성 train data 60개
        test data 800개     →   aokvqa/vqav2 외부데이터로 충당

pretrained model 3개 후보를 모두 삼성데이터 60개로 단일모델성능 비교 → clip2모델이 0.5가 넘는 유의미한 성능을 보임. & main 학습모델로 삼음.
삼성데이터 사용하지않고 전체 aokvqa 데이터로만 clip2모델 pretrained 해서 모델이 전체적인 학습데이터 흐름을 유추할 수 있도록 사전준비 완료.
하나의 외부데이터만 써서 데이터를 합치면 삼성데이터 60 + aokvqa 200~ 이런식으로 데이터 병목현상이 일어나 주객전도가 일어남.
→삼성데이터 augmentation으로 60개에서 240으로 4배 확장+ vqav2 data 추가

3가지 훈련데이터 비율(삼성,aokvqa,vqav2)을 조절해가며 clip2 성능 비교

유의미한 성과를 보인 데이터 조합으로 soft voting ensemble

--패인분석: train data & test data domain 불일치 문제--
train accuracy & val_accuracy 는 0.8 이상 나옴. but 실제 public & private score은 0.27
domain불일치 문제를 최종적으로 극복하지 못했다.
soft voting 후보들 중 최적의 성능을 내는 모델을 찾지는 못함.(시간적 한계)

<해결전략>
학습 전에 test의 시각적/텍스트적 분포를 먼저 보고 train 데이터 필터링(유사도 높은 데이터 우선)
필요하면 synthetic data 생성으로 test 스타일 보강
최종적으로 soft-voting / ensemble로 분산 낮춤

학습 전 → Zero-shot CLIP/BLIP2로 test를 한 번만 돌림.
결과를 submission에 올리지 않고, 내부적으로만 확인.
만약 train과 차이가 크면 → 외부 데이터 구성 바꾸거나 test 유사도 높은 데이터만 필터링해 학습.

--노력했으면 하는 해결방안--
데이터 증강기법,종류 다양성 강화 → 더 다양한 reasoning/fact 기반 질문, 다른 open source vqa 데이터 활용
hard example mining → 모델이 틀린문제를 다시 학습, 실제 test 데이터의 경향성을 분석하기
좀 더 다양한 후보모델 앙상블 → 일반화 성능 향상
'''
#전체코드

#구글 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

#step1 A-OKVQA 전체 데이터를 사용한 CLIP 사전학습 코드
# 📦 필요 라이브러리 설치
!pip install -q transformers timm ftfy

# 📚 라이브러리 불러오기
import os, torch
import pandas as pd
from PIL import Image, UnidentifiedImageError
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

from torch import nn
from torch.utils.data import Dataset, DataLoader
from transformers import CLIPProcessor, CLIPModel

# ⚙️ 디바이스 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ A-OKVQA 데이터 로딩
aokvqa_df = pd.read_csv("/content/drive/MyDrive/samsung/aokvqa_df_full.csv")

# 🔀 Train/Val Split
train_df, val_df = train_test_split(
    aokvqa_df, test_size=0.05, stratify=aokvqa_df["answer"], random_state=42
)

# ✅ 데이터셋 클래스 (에러 이미지 무시)
class VQADataset(Dataset):
    def __init__(self, dataframe, processor):
        self.df = dataframe.reset_index(drop=True)
        self.processor = processor
        self.label_map = {"A": 0, "B": 1, "C": 2, "D": 3}

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        try:
            row = self.df.iloc[idx]
            image_path = row["img_path"]

            if not os.path.exists(image_path):
                return None  # 파일 없으면 무시

            image = Image.open(image_path).convert("RGB")
            question = row["Question"]
            options = [row["A"], row["B"], row["C"], row["D"]]
            label = self.label_map[row["answer"]]
            texts = [f"{question} {opt}" for opt in options]

            inputs = self.processor(
                images=[image]*4,
                text=texts,
                padding="max_length",
                truncation=True,
                max_length=77,
                return_tensors="pt"
            )
            return {
                "input_ids": inputs["input_ids"],
                "attention_mask": inputs["attention_mask"],
                "pixel_values": inputs["pixel_values"],
                "label": torch.tensor(label)
            }
        except (OSError, UnidentifiedImageError):
            return None  # 깨진 이미지 무시

# ✅ collate_fn (None 제거용)
def collate_fn_remove_none(batch):
    batch = [x for x in batch if x is not None]
    if not batch:
        return None
    return {
        "input_ids": torch.stack([x["input_ids"] for x in batch]),
        "attention_mask": torch.stack([x["attention_mask"] for x in batch]),
        "pixel_values": torch.stack([x["pixel_values"] for x in batch]),
        "label": torch.stack([x["label"] for x in batch])
    }

# ✅ 모델 정의
class CLIPForMCQ(nn.Module):
    def __init__(self):
        super().__init__()
        self.clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
        self.logit_scale = nn.Parameter(torch.ones([]) * self.clip.config.logit_scale_init_value)

    def forward(self, input_ids, attention_mask, pixel_values, label=None):
        b = input_ids.shape[0]
        logits = []
        for i in range(4):
            out = self.clip(
                input_ids=input_ids[:, i],
                attention_mask=attention_mask[:, i],
                pixel_values=pixel_values[:, i]
            )
            text_feat = out.text_embeds
            image_feat = out.image_embeds
            sims = torch.cosine_similarity(text_feat, image_feat, dim=-1)
            logits.append(sims.unsqueeze(1))
        logits = torch.cat(logits, dim=1)

        if label is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, label)
            return loss, logits
        else:
            return logits

# ✅ Processor, Dataset, Dataloader
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
train_dataset = VQADataset(train_df, processor)
val_dataset = VQADataset(val_df, processor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=collate_fn_remove_none)
val_loader = DataLoader(val_dataset, batch_size=32, collate_fn=collate_fn_remove_none)

# ✅ 모델, 옵티마이저
model = CLIPForMCQ().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# ✅ 학습 루프
EPOCHS = 3

for epoch in range(1, EPOCHS + 1):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc=f"🔁 Epoch {epoch}/{EPOCHS} - Training"):
        if batch is None:
            continue
        for k in batch:
            if k != "label":
                batch[k] = batch[k].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()
        loss, _ = model(batch["input_ids"], batch["attention_mask"], batch["pixel_values"], labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"🧮 Epoch {epoch} - Train Loss: {total_loss / len(train_loader):.4f}")

    # 🔍 Validation
    model.eval()
    preds, trues = [], []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="🔍 Validation"):
            if batch is None:
                continue
            for k in batch:
                if k != "label":
                    batch[k] = batch[k].to(device)
            logits = model(batch["input_ids"], batch["attention_mask"], batch["pixel_values"])
            preds += torch.argmax(logits, dim=1).cpu().tolist()
            trues += batch["label"].tolist()

    acc = accuracy_score(trues, preds)
    f1 = f1_score(trues, preds, average="macro")
    print(f"✅ Val Accuracy: {acc:.4f} | Macro F1: {f1:.4f}")

#데이터 format 변환 및 상위 데이터추출
# ✅ 1. 패키지 설치 및 import
!pip install -q sentence-transformers

import json
import pandas as pd
import os
from collections import Counter
import random
import zipfile
import re
import torch
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm

# ✅ 2. 경로 설정
questions_path = "/content/drive/MyDrive/samsung/v2_Questions_Train_mscoco.zip"
annotations_path = "/content/drive/MyDrive/samsung/v2_Annotations_Train_mscoco.zip"
image_root = "/content/drive/MyDrive/samsung/coco2014/train2014"
samsung_csv_path = "/content/drive/MyDrive/samsung/train.csv"
converted_output = "/content/drive/MyDrive/samsung/vqav2_4choice_converted_from2014.csv"
filtered_output = "/content/drive/MyDrive/samsung/vqav2_filtered_step2_top360.csv"  # ✅ 저장 이름 변경

# ✅ 3. JSON 불러오기 함수
def load_json_from_zip(zip_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        json_file = [name for name in zip_ref.namelist() if name.endswith(".json")][0]
        with zip_ref.open(json_file) as f:
            return json.load(f)

questions_json = load_json_from_zip(questions_path)
annotations_json = load_json_from_zip(annotations_path)

# ✅ 4. 4지선다로 변환
q_dict = {q['question_id']: q for q in questions_json['questions']}
a_dict = {a['question_id']: a for a in annotations_json['annotations']}

data = []

for qid in q_dict:
    q = q_dict[qid]
    a = a_dict.get(qid, None)
    if a is None: continue

    answers = [ans["answer"] for ans in a["answers"]]
    most_common = Counter(answers).most_common()
    if not most_common: continue

    correct = most_common[0][0]
    distractors = [ans for ans, _ in most_common[1:] if ans != correct]
    distractors = list(dict.fromkeys(distractors))

    if len(distractors) < 3: continue

    choices = [correct] + distractors[:3]
    random.shuffle(choices)
    try:
        answer_letter = ['A', 'B', 'C', 'D'][choices.index(correct)]
    except ValueError:
        continue

    image_file = f"COCO_train2014_{str(q['image_id']).zfill(12)}.jpg"
    image_path = os.path.join(image_root, image_file)

    data.append({
        "Question": q["question"],
        "A": choices[0],
        "B": choices[1],
        "C": choices[2],
        "D": choices[3],
        "answer": answer_letter,
        "img_path": image_path
    })

df = pd.DataFrame(data)
df.to_csv(converted_output, index=False)
print(f"✅ 변환 완료: 총 {len(df)}개 → {converted_output}")

# ✅ 5. 유사도 기반 필터링
question_weight = 0.7
image_weight = 0.3
top_n = 360  # ✅ 여기만 변경!

samsung_df = pd.read_csv(samsung_csv_path)
vqav2_df = pd.read_csv(converted_output)

def is_valid_choice(choice):
    if pd.isna(choice) or len(choice.strip()) < 1:
        return False
    if len(choice.strip()) > 50:
        return False
    if re.fullmatch(r"[^\w\s]+", choice):
        return False
    return True

def valid_choices(row):
    return all(is_valid_choice(row[c]) for c in ['A', 'B', 'C', 'D']) and len(set([row[c] for c in ['A', 'B', 'C', 'D']])) == 4

vqav2_df = vqav2_df[vqav2_df.apply(valid_choices, axis=1)].copy()
vqav2_df = vqav2_df.drop_duplicates(subset=["Question"])

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

samsung_q_embeddings = model.encode(samsung_df["Question"].tolist(), convert_to_tensor=True, show_progress_bar=True)
samsung_img_embeddings = model.encode(samsung_df["img_path"].astype(str).tolist(), convert_to_tensor=True)

vqav2_q_embeddings = model.encode(vqav2_df["Question"].tolist(), convert_to_tensor=True, show_progress_bar=True)
vqav2_img_embeddings = model.encode(vqav2_df["img_path"].astype(str).tolist(), convert_to_tensor=True)

question_sim = util.cos_sim(vqav2_q_embeddings, samsung_q_embeddings).max(dim=1).values
image_sim = util.cos_sim(vqav2_img_embeddings, samsung_img_embeddings).max(dim=1).values

combined_score = question_sim * question_weight + image_sim * image_weight
vqav2_df["similarity_score"] = combined_score.cpu().numpy()

filtered_df = vqav2_df.sort_values(by="similarity_score", ascending=False).head(top_n)
filtered_df.to_csv(filtered_output, index=False)
print(f"✅ 최종 필터링 완료: {top_n}개 → {filtered_output}")

#삼성데이터 증강
from torchvision import transforms
import random
from torch.utils.data import Dataset
from PIL import Image
import torch
class AugmentedMCQDataset(Dataset):
    def __init__(self, df, processor, augment_ratio=2):
        self.df = df.reset_index(drop=True)
        self.processor = processor
        self.augment_ratio = augment_ratio  # 1이면 원본만, 2면 원본+증강1, 3이면 원본+증강2 등

        self.image_transform = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
            transforms.Resize((224, 224)),
        ])

    def __len__(self):
        return len(self.df) * self.augment_ratio

    def __getitem__(self, idx):
        base_idx = idx % len(self.df)
        aug_idx = idx // len(self.df)

        row = self.df.iloc[base_idx]
        try:
            image = Image.open(row['img_path']).convert("RGB")
        except Exception as e:
            return self.__getitem__((idx + 1) % self.__len__())

        # 이미지 증강
        if aug_idx > 0:
            image = self.image_transform(image)

        # 선택지 증강 (shuffle)
        choices = [row['A'], row['B'], row['C'], row['D']]
        answer_idx = ord(row['answer']) - ord('A')
        answer_text = choices[answer_idx]

        if aug_idx > 0:
            zipped = list(enumerate(choices))
            random.shuffle(zipped)
            new_choices = [c for _, c in zipped]
            new_answer_idx = new_choices.index(answer_text)
        else:
            new_choices = choices
            new_answer_idx = answer_idx

        question = row['Question']
        texts = [f"{question} {choice}" for choice in new_choices]

        inputs = self.processor(
            text=texts,
            images=[image] * 4,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=77
        )

        return {
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"],
            "pixel_values": inputs["pixel_values"],
            "label": torch.tensor(new_answer_idx)
        }

# 삼성 240 + aokvqa 240 +vqav2 360 step2모델학습

import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, ConcatDataset, random_split
from torchvision import transforms
from PIL import Image
import random
from transformers import CLIPProcessor, CLIPModel
import torch.nn as nn
from torch.optim import AdamW
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

# ✅ 저장 경로
save_best_path = "/content/drive/MyDrive/samsung/clip_best_step2_custom_aok240_vqa360.pth"
save_final_path = "/content/drive/MyDrive/samsung/clip_final_step2_custom_aok240_vqa360.pth"

device = "cuda" if torch.cuda.is_available() else "cpu"

# ✅ 데이터셋 정의
class AugmentedMCQDataset(Dataset):
    def __init__(self, df, processor, augment_ratio=4):
        self.df = df.reset_index(drop=True)
        self.processor = processor
        self.augment_ratio = augment_ratio
        self.image_transform = transforms.Compose([
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
            transforms.Resize((224, 224)),
        ])

    def __len__(self):
        return len(self.df) * self.augment_ratio

    def __getitem__(self, idx):
        base_idx = idx % len(self.df)
        aug_idx = idx // len(self.df)
        row = self.df.iloc[base_idx]

        try:
            image = Image.open(row['img_path']).convert("RGB")
        except Exception:
            return self.__getitem__((idx + 1) % self.__len__())

        if aug_idx > 0:
            image = self.image_transform(image)

        choices = [row['A'], row['B'], row['C'], row['D']]
        answer_idx = ord(row['answer']) - ord('A')
        answer_text = choices[answer_idx]

        if aug_idx > 0:
            zipped = list(enumerate(choices))
            random.shuffle(zipped)
            new_choices = [c for _, c in zipped]
            new_answer_idx = new_choices.index(answer_text)
        else:
            new_choices = choices
            new_answer_idx = answer_idx

        question = row['Question']
        texts = [f"{question} {choice}" for choice in new_choices]

        inputs = self.processor(
            text=texts,
            images=[image] * 4,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=77
        )

        return {
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"],
            "pixel_values": inputs["pixel_values"],
            "label": torch.tensor(new_answer_idx)
        }

class MCQDataset(Dataset):
    def __init__(self, df, processor):
        self.df = df.reset_index(drop=True)
        self.processor = processor

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        try:
            image = Image.open(row['img_path']).convert("RGB")
        except Exception:
            return self.__getitem__((idx + 1) % self.__len__())

        choices = [row['A'], row['B'], row['C'], row['D']]
        answer_idx = ord(row['answer']) - ord('A')
        question = row['Question']
        texts = [f"{question} {choice}" for choice in choices]

        inputs = self.processor(
            text=texts,
            images=[image] * 4,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=77
        )

        return {
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"],
            "pixel_values": inputs["pixel_values"],
            "label": torch.tensor(answer_idx)
        }

# ✅ Collate 함수
def collate_fn(batch):
    batch = [b for b in batch if b is not None]
    return {
        "input_ids": torch.stack([b["input_ids"] for b in batch]),
        "attention_mask": torch.stack([b["attention_mask"] for b in batch]),
        "pixel_values": torch.stack([b["pixel_values"] for b in batch]),
        "label": torch.tensor([b["label"] for b in batch])
    }

# ✅ 데이터 로드
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

samsung_df = pd.read_csv("/content/drive/MyDrive/samsung/train.csv")
samsung_df["img_path"] = samsung_df["img_path"].apply(lambda x: os.path.join("/content/drive/MyDrive/samsung/train_input_images/", os.path.basename(x)))

# ✅ A-OKVQA
aokvqa_df = pd.read_csv("/content/drive/MyDrive/samsung/aokvqa_step2_top240_weighted.csv")
aokvqa_df["img_path"] = aokvqa_df["img_path"].apply(
    lambda x: os.path.join("/content/drive/MyDrive/samsung/A-OKVQA/images/train2017", os.path.basename(x))
)

# ✅ VQAv2
vqav2_df = pd.read_csv("/content/drive/MyDrive/samsung/vqav2_filtered_step2_top360.csv")
vqav2_df["img_path"] = vqav2_df["img_path"].apply(lambda x:
    os.path.join("/content/drive/MyDrive/samsung/coco2014/train2014", os.path.basename(x)))


samsung_dataset = AugmentedMCQDataset(samsung_df, processor, augment_ratio=4)
aokvqa_dataset = MCQDataset(aokvqa_df, processor)
vqav2_dataset = MCQDataset(vqav2_df, processor)

full_dataset = ConcatDataset([samsung_dataset, aokvqa_dataset, vqav2_dataset])
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, collate_fn=collate_fn)

# ✅ Early Stopping 설정
early_stopping_counter = 0
patience = 2

# ✅ 모델 정의
class CLIPForMCQ(nn.Module):
    def __init__(self):
        super().__init__()
        self.clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
        self.classifier = nn.Linear(self.clip.config.projection_dim, 1)

    def forward(self, input_ids, attention_mask, pixel_values):
        b, c, _ = input_ids.shape
        input_ids = input_ids.view(-1, input_ids.size(-1))
        attention_mask = attention_mask.view(-1, attention_mask.size(-1))
        pixel_values = pixel_values.view(-1, *pixel_values.shape[2:])

        text_feat = self.clip.get_text_features(input_ids=input_ids, attention_mask=attention_mask)
        image_feat = self.clip.get_image_features(pixel_values=pixel_values)
        logits = self.classifier(text_feat * image_feat).view(b, c)
        return logits

# ✅ 학습
model = CLIPForMCQ().to(device)
model.clip.load_state_dict(torch.load("/content/drive/MyDrive/samsung/clip_pretrain_step1.pth", map_location=device))

optimizer = AdamW(model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()
best_f1 = 0

# 🔁 학습 루프
for epoch in range(5):
    model.train()
    total_loss, preds, labels = 0, [], []

    for batch in tqdm(train_loader, desc=f"🧪 Train Epoch {epoch+1}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        pixel_values = batch["pixel_values"].to(device)
        target = batch["label"].to(device)

        logits = model(input_ids, attention_mask, pixel_values)
        loss = criterion(logits, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        preds += torch.argmax(logits, dim=1).cpu().tolist()
        labels += target.cpu().tolist()

    train_f1 = f1_score(labels, preds, average="macro")
    train_acc = accuracy_score(labels, preds)
    train_loss = total_loss / len(train_loader)

    model.eval()
    val_preds, val_labels = [], []
    with torch.no_grad():
        for batch in tqdm(val_loader, desc=f"🔍 Val Epoch {epoch+1}"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            pixel_values = batch["pixel_values"].to(device)
            target = batch["label"].to(device)

            logits = model(input_ids, attention_mask, pixel_values)
            val_preds += torch.argmax(logits, dim=1).cpu().tolist()
            val_labels += target.cpu().tolist()

    val_f1 = f1_score(val_labels, val_preds, average="macro")
    val_acc = accuracy_score(val_labels, val_preds)

    print(f"\n📊 Epoch {epoch+1}")
    print(f"  🏋️‍♂️ Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f}, F1: {train_f1:.4f}")
    print(f"  🔍 Val   Acc: {val_acc:.4f}, F1: {val_f1:.4f}")

    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.clip.state_dict(), save_best_path)
        early_stopping_counter = 0
    else:
        early_stopping_counter += 1
        if early_stopping_counter >= patience:
            print("⛔ Early stopping triggered!")
            break

#테스트 data 학습 및 submission
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from transformers import CLIPProcessor, CLIPModel
import torch.nn as nn
from tqdm import tqdm
import os

# ✅ 경로 설정
test_csv_path = "/content/drive/MyDrive/samsung/test.csv"
image_dir = "/content/drive/MyDrive/samsung/test_input_images"
model_path_1 = "/content/drive/MyDrive/samsung/clip_best_step2_custom_aok240_vqa360.pth"  # 가중치 0.7
model_path_2 = "/content/drive/MyDrive/samsung/clip_best_step2_total.pth"                # 가중치 0.3
submission_path = "/content/drive/MyDrive/samsung/submission_soft.csv"

device = "cuda" if torch.cuda.is_available() else "cpu"

# ✅ 1. Dataset 정의
class TestDataset(Dataset):
    def __init__(self, df, processor):
        self.df = df.reset_index(drop=True)
        self.processor = processor

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = os.path.join(image_dir, os.path.basename(row["img_path"]))
        image = Image.open(image_path).convert("RGB")

        choices = [row["A"], row["B"], row["C"], row["D"]]
        texts = [f"{row['Question']} {c}" for c in choices]

        inputs = self.processor(
            text=texts,
            images=[image] * 4,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=77
        )

        return {
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"],
            "pixel_values": inputs["pixel_values"],
            "ID": row["ID"]
        }

# ✅ 2. 모델 정의 (clip만 따로 불러올 수 있게)
class CLIPForMCQ(nn.Module):
    def __init__(self):
        super().__init__()
        self.clip = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
        self.classifier = nn.Linear(self.clip.config.projection_dim, 1)

    def forward(self, input_ids, attention_mask, pixel_values):
        b, c, _ = input_ids.shape
        input_ids = input_ids.view(-1, input_ids.size(-1))
        attention_mask = attention_mask.view(-1, attention_mask.size(-1))
        pixel_values = pixel_values.view(-1, *pixel_values.shape[2:])

        text_feat = self.clip.get_text_features(input_ids=input_ids, attention_mask=attention_mask)
        image_feat = self.clip.get_image_features(pixel_values=pixel_values)
        logits = self.classifier(text_feat * image_feat).view(b, c)
        return logits

# ✅ 3. 데이터 로드
test_df = pd.read_csv(test_csv_path)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
test_dataset = TestDataset(test_df, processor)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# ✅ 4. 모델 두 개 로드
def load_model_weights(model_path):
    model = CLIPForMCQ().to(device)
    model.clip.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    return model

model1 = load_model_weights(model_path_1)  # 0.7
model2 = load_model_weights(model_path_2)  # 0.3

# ✅ 5. Soft Voting Inference
id_list, pred_list = [], []
with torch.no_grad():
    for batch in tqdm(test_loader, desc="🔍 Soft Voting Predicting"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        pixel_values = batch["pixel_values"].to(device)

        logits1 = model1(input_ids, attention_mask, pixel_values)
        logits2 = model2(input_ids, attention_mask, pixel_values)

        # 7:3 Soft Voting
        final_logits = logits1 * 0.7 + logits2 * 0.3
        preds = torch.argmax(final_logits, dim=1).cpu().tolist()

        id_list += batch["ID"]
        pred_list += [chr(ord("A") + p) for p in preds]

# ✅ 6. 제출 파일 생성
submission_df = pd.DataFrame({"ID": id_list, "answer": pred_list})
submission_df.to_csv(submission_path, index=False)
print(f"✅ Soft Voting 제출 파일 저장 완료 → {submission_path}")

